In [5]:
import os
import dspy


In [6]:
import dspy
lm = dspy.LM("openai/gpt-4o-mini")
dspy.configure(lm=lm)

CHECK FOR QUESTION COMPLETENESS

In [70]:
class CheckQuestionCompleteness(dspy.Signature):
    """
    Determines whether a question is complete and understandable on its own.
    A question is considered complete if it meets any of these:
    - It contains both a subject (explicit or implied, e.g., 'you', 'they') and an object (what is being asked or acted upon).
    - It is a composite question (a single sentence containing multiple understandable sub-questions), where all sub-questions have subjects and objects.
    If the question’s meaning is clear, even with an implied subject or multiple sub-questions, mark it as complete.
    """
    question: str = dspy.InputField(desc="The question to evaluate for completeness.")
    completeness: bool = dspy.OutputField(desc="True if the question is complete, otherwise False.")
    reason: str = dspy.OutputField(desc="Brief explanation of why the question is complete or incomplete.")




query_entirety_checker = dspy.Predict(CheckQuestionCompleteness)

In [71]:
questions = [
    "What is the ?",
    "Is the Tomato Ketchup healthy for Children",
    "can you track",
    "Do you offer discounts in the festive season",
    "Do you sell baby diapers, are any size variations, do you sell them at discounts?"
    
]


for q in questions:
    result = query_entirety_checker(question=q)
    print(f"Question: {q}")
    print(f"Completeness: {result.completeness}")
    print(f"Reason: {result.reason}")
    print("-" * 50)

Question: What is the ?
Completeness: False
Reason: The question is incomplete because it lacks a subject and an object, making it unclear what is being asked.
--------------------------------------------------
Question: Is the Tomato Ketchup healthy for Children
Completeness: False
Reason: The question lacks a clear subject; it does not specify who is asking or imply a subject. While it asks about the healthiness of tomato ketchup for children, it does not provide a complete context or a clear subject performing the action.
--------------------------------------------------
Question: can you track
Completeness: False
Reason: The question "can you track" is incomplete because it lacks a clear subject and object. It does not specify what is being tracked or who is being asked to track, making it ambiguous and unclear.
--------------------------------------------------
Question: Do you offer discounts in the festive season
Completeness: True
Reason: The question is complete because it ha

DECOMPOSE THE QUESTION TO COMPONENT QUESTIONS

In [47]:

class BreakCompositeQuestion(dspy.Signature):
    """
    You are a question decomposition expert.
    
    Your task is to break down complex, multi-part questions into smaller, atomic questions that can be answered independently.
    
    However, if the input question is already simple, direct, and self-contained — meaning it asks only one thing and does not contain multiple clauses or conjunctions — return it as-is in a single-item list.
    
    Do not infer or expand the question. Do not add subtext or assumptions. Do not rephrase.
    
    Examples:
    - Input: "Do you offer discount?" → Output: ["Do you offer discount?"]
    - Input: "What are your prices and do you offer discounts?" → Output: ["What are your prices?", "Do you offer discounts?"]
    - Input: "How does the product work?" → Output: ["How does the product work?"]
    - Input: "How does the product work and what are its benefits?" → Output: ["How does the product work?", "What are its benefits?"]
    """
    question: str = dspy.InputField(desc="The composite question to be broken down.")
    sub_questions: list[str] = dspy.OutputField(desc="List of smaller, atomic questions derived from the input.")



breakdown_model = dspy.Predict(BreakCompositeQuestion)




In [48]:

composite_question = "Do you sell Atta noodles, is it nutritious, tell me whether it is a refined product or not ?"


result = breakdown_model(question=composite_question)


print("Original Question:\n", composite_question)
print("\nDecomposed Questions:")
for i, q in enumerate(result.sub_questions, start=1):
    print(f"{i}. {q}")

Original Question:
 Do you sell Atta noodles, is it nutritious, tell me whether it is a refined product or not ?

Decomposed Questions:
1. Do you sell Atta noodles?
2. Is it nutritious?
3. Is it a refined product or not?


In [49]:

composite_question = "Do you offer discounts in festive season?"


result = breakdown_model(question=composite_question)


print("Original Question:\n", composite_question)
print("\nDecomposed Questions:")
for i, q in enumerate(result.sub_questions, start=1):
    print(f"{i}. {q}")

Original Question:
 Do you offer discounts in festive season?

Decomposed Questions:
1. Do you offer discounts in festive season?


CHECK WHETHER THE QUESTION IS RELEVANT OR NOT

In [14]:

class CheckEcommerceRelevance(dspy.Signature):
    """
    You are an intelligent assistant that determines whether a question
    is relevant to an e-commerce store's domain.

    E-commerce relevance includes:
    - Products, pricing, reviews, discounts
    - Orders, carts, shipping, delivery
    - Payments, refunds, account issues
    - Customer support, tracking, and loyalty programs

    Irrelevant topics include:
    - Politics, personal life, unrelated tech, or general knowledge
    """
    question: str = dspy.InputField(desc="The user question to evaluate.")
    relevance: bool = dspy.OutputField(desc="True if relevant to e-commerce, otherwise False.")
    reason: str = dspy.OutputField(desc="Explain briefly why the question is or isn’t relevant.")


ecommerce_relevance_checker = dspy.Predict(CheckEcommerceRelevance)




In [15]:
questions = [
    "What is the refund policy for defective items?",
    "Who won the cricket match yesterday?",
    "Do you have discounts on smartphones?",
    "Explain quantum computing in simple terms.",
    "How can I track my order?",
    "Who is Finance Minister of India?"
]


for q in questions:
    result = ecommerce_relevance_checker(question=q)
    print(f"Question: {q}")
    print(f"Relevance: {result.relevance}")
    print(f"Reason: {result.reason}")
    print("-" * 50)

Question: What is the refund policy for defective items?
Relevance: True
Reason: The question is relevant to e-commerce as it pertains to the refund policy, which is a key aspect of customer service and order management in online shopping.
--------------------------------------------------
Question: Who won the cricket match yesterday?
Relevance: False
Reason: The question is about a sports event and does not pertain to e-commerce topics such as products, orders, or customer support.
--------------------------------------------------
Question: Do you have discounts on smartphones?
Relevance: True
Reason: The question is relevant to e-commerce as it inquires about discounts on a specific product category, which is smartphones. Discounts are a common aspect of e-commerce that customers are interested in.
--------------------------------------------------
Question: Explain quantum computing in simple terms.
Relevance: False
Reason: The question about quantum computing is not relevant to e

IDENTIFY THE QUERY CATEGORY

In [17]:
class ClassifyEcommerceQuery(dspy.Signature):
    """
    You are an intelligent assistant that classifies a user's question
    into specific e-commerce-related categories.

    Possible categories include (but are not limited to):
    - Product Inquiry: product details, specifications, availability, reviews
    - Order Tracking: order status, shipment updates, delivery time
    - Return/Refund: product returns, cancellations, refund status
    - Payment/Billing: payment issues, discounts, failed transactions
    - Account/Support: login issues, account settings, loyalty programs
    - General Inquiry: store policies, offers, customer service, etc.

    Always choose the category that best matches the user’s intent.
    If uncertain, pick the closest relevant category.
    """
    question: str = dspy.InputField(desc="The user question to classify.")
    category: str = dspy.OutputField(desc="The most relevant e-commerce category for this question.")
    reason: str = dspy.OutputField(desc="Briefly explain why the question fits this category.")

ecommerce_query_type = dspy.Predict(ClassifyEcommerceQuery)

In [19]:
questions = [
    "What is the refund policy for defective items?",
    "Do you have discounts on smartphones?",
    "How can I track my order?",
    
]


for q in questions:
    result = ecommerce_query_type(question=q)
    print(f"Question: {q}")
    print(f"Category: {result.category}")
    print(f"Reason: {result.reason}")
    print("-" * 50)

Question: What is the refund policy for defective items?
Category: Return/Refund
Reason: The question pertains to the policy regarding refunds for defective items, which directly relates to the return and refund process.
--------------------------------------------------
Question: Do you have discounts on smartphones?
Category: General Inquiry
Reason: The question is about discounts, which falls under general inquiries regarding store offers and promotions rather than a specific product or transaction issue.
--------------------------------------------------
Question: How can I track my order?
Category: Order Tracking
Reason: The question is about tracking an order, which directly relates to checking the status and updates of a shipment.
--------------------------------------------------


COMPLETE DSPY PIPELINE

In [72]:

class EcommercePipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.check_completeness = dspy.Predict(CheckQuestionCompleteness)
        self.break_question = dspy.Predict(BreakCompositeQuestion)
        self.check_relevance = dspy.Predict(CheckEcommerceRelevance)
        self.classify_query = dspy.Predict(ClassifyEcommerceQuery)

    def forward(self, question: str):
        # Step 1: Completeness check
        completeness_result = self.check_completeness(question=question)
        if not completeness_result.completeness:
            return {
                "complete_status": "incomplete question",
                "complete_reason": completeness_result.reason
            }

        # Step 2: Break composite question
        sub_q_result = self.break_question(question=question)
        sub_questions = sub_q_result.sub_questions

        results = []
        for q in sub_questions:
            # Step 3: Relevance check
            relevance_result = self.check_relevance(question=q)
            if not relevance_result.relevance:
                continue

            # Step 4: Classification
            category_result = self.classify_query(question=q)
            results.append({
                "question": q,
                "category": category_result.category,
                "category_reason": category_result.reason
            })

        if not results:
            return {"status": "no_relevant_questions"}

        return {
            "status": "success",
            "processed_questions": results
        }
pipeline = EcommercePipeline()

In [63]:


response = pipeline("What is?")
print(f"Complete Status: {response['complete_status']}")
print(f"Complete Reason: {response['complete_reason']}")

Complete Status: incomplete question
Complete Reason: The question "What is?" is incomplete because it lacks a clear subject and object. It does not specify what is being asked about, making it ambiguous and unclear.


In [76]:
response = pipeline("Do you sell baby diapers, are any size variations, do you sell them at discounts?")
pprint.pprint(response)

{'processed_questions': [{'category': 'Product Inquiry',
                          'category_reason': 'The question is asking about the '
                                             'availability of a specific '
                                             'product, which in this case is '
                                             'baby diapers. This fits the '
                                             'Product Inquiry category as it '
                                             'relates to product details.',
                          'question': 'Do you sell baby diapers?'},
                         {'category': 'Product Inquiry',
                          'category_reason': 'The question is asking about '
                                             'size variations, which pertains '
                                             'to product details and '
                                             'specifications.',
                          'question': 'Are there any size va